In [25]:
import tensorflow as tf
import os

class JerkModelWindowed(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec([4, None], tf.float32)])
    def __call__(self, inputs):
        timestamps = inputs[0]
        x = inputs[1]
        y = inputs[2]
        z = inputs[3]

        # Derivada centrada para jerk
        x_diff = (x[2:] - x[:-2]) / ((timestamps[2:] - timestamps[:-2]) / 2.0)
        y_diff = (y[2:] - y[:-2]) / ((timestamps[2:] - timestamps[:-2]) / 2.0)
        z_diff = (z[2:] - z[:-2]) / ((timestamps[2:] - timestamps[:-2]) / 2.0)

        jerk = tf.sqrt(x_diff**2 + y_diff**2 + z_diff**2)
        t_valid = timestamps[1:-1]

        win = tf.cast(tf.floor(t_valid / 5000.0), tf.int32)
        min_win = tf.reduce_min(win)
        win0 = win - min_win
        num_wins = tf.reduce_max(win0) + 1

        sums = tf.math.unsorted_segment_sum(jerk, win0, num_wins)
        counts = tf.math.unsorted_segment_sum(tf.ones_like(jerk), win0, num_wins)
        means = sums / counts
        valid = counts > 0

        num_valid = tf.reduce_sum(tf.cast(valid, tf.float32))
        sum_means = tf.reduce_sum(tf.where(valid, means, 0.0))
        sum_sq = tf.reduce_sum(tf.where(valid, means**2, 0.0))

        mean_of_means = sum_means / num_valid
        std_dev = tf.sqrt((sum_sq / num_valid) - mean_of_means**2)

        min_val = tf.reduce_min(tf.where(valid, means, 1e30))
        max_val = tf.reduce_max(tf.where(valid, means, -1e30))

        return tf.stack([mean_of_means, std_dev, min_val, max_val])

# Salvar modelo corrigido
model = JerkModelWindowed()
concrete_fn = model.__call__.get_concrete_function()
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_fn], model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
tflite_model = converter.convert()

with open("jerk_model_windowed.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Modelo regenerado com input em float32.")

INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp56n36_4s/assets


INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp56n36_4s/assets


✅ Modelo regenerado com input em float32.


W0000 00:00:1750713692.491968 22810823 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1750713692.491985 22810823 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-23 18:21:32.492142: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp56n36_4s
2025-06-23 18:21:32.492407: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-23 18:21:32.492414: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp56n36_4s
2025-06-23 18:21:32.495053: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-23 18:21:32.500676: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp56n36_4s
2025-06-23 18:21:32.506089: I tensorflow/cc/saved_model/loader.c

In [1]:
import numpy as np
import tensorflow as tf
import time
import os

# --- Função de referência em Python puro ---
def jerk_by_window_py(sensor_data, window_size_ms=5000):
    timestamps = sensor_data[0]
    x = sensor_data[1].astype(np.float32)
    y = sensor_data[2].astype(np.float32)
    z = sensor_data[3].astype(np.float32)

    N = len(timestamps)
    jerk_values = []

    for i in range(1, N - 1):
        dt = (timestamps[i + 1] - timestamps[i - 1]) / 2.0  # delta médio
        if dt == 0:
            jerk = 0.0
        else:
            dx = (x[i + 1] - x[i - 1]) / dt
            dy = (y[i + 1] - y[i - 1]) / dt
            dz = (z[i + 1] - z[i - 1]) / dt
            jerk = np.sqrt(dx**2 + dy**2 + dz**2)
        jerk_values.append((timestamps[i], jerk))

    jerk_values = np.array(jerk_values, dtype=np.float32)

    window_ids = np.floor(jerk_values[:, 0] / window_size_ms).astype(np.int32)
    unique_ids = np.unique(window_ids)

    window_means = []
    for wid in unique_ids:
        values = jerk_values[window_ids == wid, 1]
        if len(values) > 0:
            window_means.append(np.mean(values))

    window_means = np.array(window_means)
    return np.array([
        np.mean(window_means),
        np.std(window_means),
        np.min(window_means),
        np.max(window_means)
    ], dtype=np.float32)

# --- Geração de dados simulados ---
N = 10000
timestamps = np.arange(N, dtype=np.int32) * 20  # 20 ms de intervalo
x = np.random.rand(N).astype(np.float32) * 10
y = np.random.rand(N).astype(np.float32) * 10
z = np.random.rand(N).astype(np.float32) * 10
sensor_data = np.stack([timestamps, x, y, z], axis=0).astype(np.float32)  # shape [4, N]

# --- Cálculo em Python ---
start_cpu = time.time()
jerk_ref = jerk_by_window_py(sensor_data.copy())
cpu_time_ms = (time.time() - start_cpu) * 1000

# --- Inferência TFLite ---
model_path = "jerk_model_windowed.tflite"
start_load = time.time()
interpreter = tf.lite.Interpreter(model_path=model_path)
end_load = time.time()
load_time_ms = (end_load - start_load) * 1000

interpreter.resize_tensor_input(0, [4, N])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

start_tflite = time.time()
interpreter.set_tensor(input_details[0]['index'], sensor_data)
interpreter.invoke()
jerk_out = interpreter.get_tensor(output_details[0]['index'])[0]
end_tflite = time.time()
tflite_time_ms = (end_tflite - start_tflite) * 1000

# --- Estatísticas ---
erro_abs = np.abs(jerk_ref - jerk_out)
reduction_pct = ((cpu_time_ms - tflite_time_ms) / cpu_time_ms) * 100
model_size_kb = os.path.getsize(model_path) / 1024

# --- Resultados ---
print("📊 Resultado:")
print("Referência (Python):", jerk_ref)
print("Modelo TFLite      :", jerk_out)
print("Erro absoluto      :", erro_abs)
print(f"\n⏱️ Tempo CPU:     {cpu_time_ms:.2f} ms")
print(f"⚡ Tempo TFLite:   {tflite_time_ms:.2f} ms")
print(f"📉 Redução tempo : {reduction_pct:.2f}%")
print(f"💾 Tamanho modelo : {model_size_kb:.2f} KB")

/Users/joaovitorcotta/StudioProjects/VulkanFFT/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


📊 Resultado:
Referência (Python): [0.32929283 0.00769046 0.30671316 0.34594452]
Modelo TFLite      : 0.32929283
Erro absoluto      : [0.         0.32160237 0.02257967 0.01665169]

⏱️ Tempo CPU:     22.90 ms
⚡ Tempo TFLite:   0.25 ms
📉 Redução tempo : 98.92%
💾 Tamanho modelo : 9.43 KB


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
